In [1]:
import os

import numpy as np
import vtk
from vtk.util.numpy_support import numpy_to_vtk, vtk_to_numpy

import pyfocusr
from pyfocusr.vtk_functions import read_vtk_mesh

In [2]:
location_meshes = '../data/'
n_points = '5k'

if n_points == '5k':
    target_filename = 'target_mesh.vtk'
    source_filename = 'source_mesh.vtk'
elif n_points == '15k':
    target_filename = 'target_mesh_15k.vtk'
    source_filename = 'source_mesh_15k.vtk'
    
target_vtk_mesh = read_vtk_mesh(os.path.join(location_meshes, target_filename))
source_vtk_mesh = read_vtk_mesh(os.path.join(location_meshes, source_filename))

Best results yet with: 

```
reg = pyfocusr.Focusr(vtk_mesh_target=target_vtk_mesh, 
                      vtk_mesh_source=source_vtk_mesh,
                      n_spectral_features=3,
                      n_extra_spectral=3,
                      get_weighted_spectral_coords=True,
                      list_features_to_calc=['curvature'],
                      rigid_reg_max_iterations=100,
                      non_rigid_alpha=0.001,
                      non_rigid_beta=4,
                      non_rigid_n_eigens=100,
                      non_rigid_max_iterations=300,
                      rigid_before_non_rigid_reg=False,
                      projection_smooth_iterations=40,
                      graph_smoothing_iterations=300,
                      include_points_as_features=True,
                      norm_physical_and_spectral=True,
                      feature_weights=np.diag([1,1]),
                      n_coords_spectral_ordering=10000)
```

In [3]:
reg = pyfocusr.Focusr(vtk_mesh_target=target_vtk_mesh, 
                      vtk_mesh_source=source_vtk_mesh,
                      n_spectral_features=3,
                      n_extra_spectral=3,
                      get_weighted_spectral_coords=True,
                      list_features_to_calc=['curvature'],
                      rigid_reg_max_iterations=100,
                      non_rigid_alpha=0.001,
                      non_rigid_beta=4,
                      non_rigid_n_eigens=100,
                      non_rigid_max_iterations=300,
                      rigid_before_non_rigid_reg=False,
                      projection_smooth_iterations=40,
                      graph_smoothing_iterations=300,
                      include_points_as_features=True,
                      norm_physical_and_spectral=True,
                      feature_weights=np.diag([1,1]),
                      n_coords_spectral_ordering=10000,
                      n_coords_spectral_registration=1000)

Loaded Mesh 1
building adjacency matrix
building degree matrix
starting to get laplacian matrix
beginning eigen decomposition
Computed spectrum 1
Loaded Mesh 2
building adjacency matrix
building degree matrix
starting to get laplacian matrix
beginning eigen decomposition
Computed spectrum 2


### View mesh features

In [4]:
# reg.graph_source.view_mesh_features(feature_idx=1)

In [5]:
reg.graph_source.eig_vals

array([0.00075626, 0.00101274, 0.00139415, 0.0023161 , 0.00301983,
       0.00361393])

### View un-ordered/corrected/weighted eigenvectors (spectral coordinates)

#### Source Eigenvector 1

In [6]:
eig_vec = 0

In [7]:
reg.graph_source.view_mesh_eig_vec(eig_vec=eig_vec)

Viewer(geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'name': '_points', 'numberO…

#### Target Eigenvector 1

In [8]:
reg.graph_target.view_mesh_eig_vec(eig_vec=eig_vec)

Viewer(geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'name': '_points', 'numberO…

# Notes about comparison: 
Depending on if we include features ('curvature') or not, we will get different results for these meshes. 

If we leave the list at `list_features_to_calc` as an empty list `[]` then: <br>
We note that the color gradient is the same (in the same direction) between the two meshes. However, the colours are flipped. This is because the direction of "positive" can be flipped from one mesh to another. This will (should) be corrected automatically before the registration begins. 

If we include `'curvature'` in `list_features_to_calc` with `['curvature']` then: <br>
We get eigenvectors that align in direction and orientation (at least for the first 3 (0, 1, 2). 

Note, the flipped nature of the spectral coordinates can/will be corrected in the algorithm (if it exists). However, this only works because these are both of the same leg (right - as evident by the large side of the trochlear groove) and will likely faily/break if a left and right leg were analyzed. Therefore, all legs should be flipped to be the same side to "fix" things. 

## Begin Registration
This will print out the registration parameters/results as it goes. <br>
- First it will print out the "pairs" of eigenvectors between the two meshes and which ones on the source were flipped to match the target eigenvectors. 
- Second it will print the rigid registration progress and ultimately the registration "results"/"parameters"
- Third it will print the same for the non-rigid registration progress. 


In [9]:
reg.align_maps()

Eigenvector Sorting Results

The matches for eigenvectors were as follows:
Target	|  Source
     0	|  -0    
     1	|  -1    
     2	|  2     
     3	|  -3    
     4	|  -4    
     5	|  -5    
*Negative source values means those eigenvectors were flipped*

Appending Extra Features to Spectral Coords
Number of features (including spectral) used for registartion: 8

Non-Rigid (Deformable) Registration Beginning

Iteration:1
ML:-15235.952; 	ML change (error): 15235.952; 	Sigma^2:     0.015; 	Sigma^2 change:     0.061
[                                                                        ]
Iteration:2
ML:-19902.364; 	ML change (error):  4666.412; 	Sigma^2:     0.006; 	Sigma^2 change:     0.009
[                                                                        ]
Iteration:3
ML:-22419.036; 	ML change (error):  2516.672; 	Sigma^2:     0.002; 	Sigma^2 change:     0.004
[                                                                        ]
Iteration:4
ML:-25499.284; 	ML change (err

Iteration:46
ML:-31930.779; 	ML change (error):     0.317; 	Sigma^2:     0.000; 	Sigma^2 change:     0.000
[===========                                                             ]
Iteration:47
ML:-31931.073; 	ML change (error):     0.294; 	Sigma^2:     0.000; 	Sigma^2 change:    -0.000
[===========                                                             ]
Iteration:48
ML:-31931.338; 	ML change (error):     0.266; 	Sigma^2:     0.000; 	Sigma^2 change:    -0.000
[===========                                                             ]
Iteration:49
ML:-31931.593; 	ML change (error):     0.255; 	Sigma^2:     0.000; 	Sigma^2 change:    -0.000
[===========                                                             ]
Iteration:50
ML:-31931.871; 	ML change (error):     0.278; 	Sigma^2:     0.000; 	Sigma^2 change:    -0.000
[============                                                            ]
Iteration:51
ML:-31932.238; 	ML change (error):     0.366; 	Sigma^2:     0.000; 	Sigma^2 c

Iteration:97
ML:-31969.772; 	ML change (error):     0.163; 	Sigma^2:     0.000; 	Sigma^2 change:     0.000
[=======================                                                 ]
Iteration:98
ML:-31969.908; 	ML change (error):     0.135; 	Sigma^2:     0.000; 	Sigma^2 change:     0.000
[=======================                                                 ]
Iteration:99
ML:-31970.005; 	ML change (error):     0.097; 	Sigma^2:     0.000; 	Sigma^2 change:     0.000
[=======================                                                 ]
Iteration:100
ML:-31970.068; 	ML change (error):     0.063; 	Sigma^2:     0.000; 	Sigma^2 change:     0.000
[========================                                                ]
Iteration:101
ML:-31970.106; 	ML change (error):     0.038; 	Sigma^2:     0.000; 	Sigma^2 change:     0.000
[========================                                                ]
Iteration:102
ML:-31970.129; 	ML change (error):     0.023; 	Sigma^2:     0.000; 	Sigma^

Iteration:142
ML:-31970.164; 	ML change (error):     0.000; 	Sigma^2:     0.000; 	Sigma^2 change:    -0.000
[==================================                                      ]
Iteration:143
ML:-31970.164; 	ML change (error):     0.000; 	Sigma^2:     0.000; 	Sigma^2 change:    -0.000
[==================================                                      ]
Iteration:144
ML:-31970.164; 	ML change (error):     0.000; 	Sigma^2:     0.000; 	Sigma^2 change:    -0.000
[==================================                                      ]
Iteration:145
ML:-31970.164; 	ML change (error):     0.000; 	Sigma^2:     0.000; 	Sigma^2 change:    -0.000
[==================================                                      ]
Iteration:146
ML:-31970.164; 	ML change (error):     0.000; 	Sigma^2:     0.000; 	Sigma^2 change:     0.000
[===================================                                     ]
Iteration:147
ML:-31970.164; 	ML change (error):     0.000; 	Sigma^2:     0.000; 	Sig

Iteration:189
ML:-31970.164; 	ML change (error):     0.000; 	Sigma^2:     0.000; 	Sigma^2 change:    -0.000
[=============================================                           ]
Iteration:190
ML:-31970.164; 	ML change (error):     0.000; 	Sigma^2:     0.000; 	Sigma^2 change:     0.000
[=============================================                           ]
Iteration:191
ML:-31970.164; 	ML change (error):     0.000; 	Sigma^2:     0.000; 	Sigma^2 change:    -0.000
[=============================================                           ]
Iteration:192
ML:-31970.164; 	ML change (error):     0.000; 	Sigma^2:     0.000; 	Sigma^2 change:     0.000
[==============================================                          ]
Iteration:193
ML:-31970.164; 	ML change (error):     0.000; 	Sigma^2:     0.000; 	Sigma^2 change:     0.000
[==============================================                          ]
Iteration:194
ML:-31970.164; 	ML change (error):     0.000; 	Sigma^2:     0.000; 	Sig

Iteration:235
ML:-31970.164; 	ML change (error):     0.000; 	Sigma^2:     0.000; 	Sigma^2 change:     0.000
[========================================================                ]
Iteration:236
ML:-31970.164; 	ML change (error):     0.000; 	Sigma^2:     0.000; 	Sigma^2 change:     0.000
[========================================================                ]
Iteration:237
ML:-31970.164; 	ML change (error):     0.000; 	Sigma^2:     0.000; 	Sigma^2 change:     0.000
[========================================================                ]
Iteration:238
ML:-31970.164; 	ML change (error):     0.000; 	Sigma^2:     0.000; 	Sigma^2 change:    -0.000
[=========================================================               ]
Iteration:239
ML:-31970.164; 	ML change (error):     0.000; 	Sigma^2:     0.000; 	Sigma^2 change:     0.000
[=========================================================               ]
Iteration:240
ML:-31970.164; 	ML change (error):     0.000; 	Sigma^2:     0.000; 	Sig

Iteration:286
ML:-31970.164; 	ML change (error):     0.000; 	Sigma^2:     0.000; 	Sigma^2 change:     0.000
[====================================================================    ]
Iteration:287
ML:-31970.164; 	ML change (error):     0.000; 	Sigma^2:     0.000; 	Sigma^2 change:    -0.000
[====================================================================    ]
Iteration:288
ML:-31970.164; 	ML change (error):     0.000; 	Sigma^2:     0.000; 	Sigma^2 change:    -0.000
[=====================================================================   ]
Iteration:289
ML:-31970.164; 	ML change (error):     0.000; 	Sigma^2:     0.000; 	Sigma^2 change:     0.000
[=====================================================================   ]
Iteration:290
ML:-31970.164; 	ML change (error):     0.000; 	Sigma^2:     0.000; 	Sigma^2 change:     0.000
[=====================================================================   ]
Iteration:291
ML:-31970.164; 	ML change (error):     0.000; 	Sigma^2:     0.000; 	Sig

array([ 111,  112,  111, ..., 4983, 4984, 4954])

## View the spectral coordinates

In [10]:
reg.view_aligned_spectral_coords(starting_spectral_coord=0, include_rigid_aligned=True, include_unaligned=True)

Viewer(point_set_colors=array([[0.8392157 , 0.        , 0.        ],
       [0.54901963, 0.23137255, 1.       …

In [11]:
reg.view_meshes_colored_by_spectral_correspondences()

Viewer(geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'name': '_points', 'numberO…

In [12]:
from itkwidgets import view

In [13]:
view(point_sets=[reg.smoothed_target_coords, reg.source_projected_on_target])

Viewer(geometries=[], gradient_opacity=0.22, point_set_colors=array([[0.8392157 , 0.        , 0.        ],
   …

In [14]:
view(point_sets=[reg.source_projected_on_target])

Viewer(geometries=[], gradient_opacity=0.22, point_set_colors=array([[0.8392157, 0.       , 0.       ]], dtype…

In [15]:
len(np.unique(reg.corresponding_target_idx_for_each_source_pt))

3531

In [16]:
len(np.unique(reg.corresponding_target_idx_for_each_source_pt))

3531

In [17]:
reg.view_meshes_colored_by_spectral_correspondences()

Viewer(geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'name': '_points', 'numberO…